In [1]:
from fompy.constants import *
from fompy.units import unit, from_unit
from fompy.models import *
from fompy.materials import Si
from math import sqrt, exp, pi, log
from scipy.optimize import bisect

# Task 1

In [2]:
#InSb
m1=114*amu
m2=121*amu
m=(m1+m2)/2
a=6.479*angstrom
l=DiamondLikeLattice(a, m)

In [3]:
print(f'Density: {l.density}') # g / cm^3
print(f'Concentration: {l.concentration}') # g / cm^3

Density: 5.7408149263833534
Concentration: 2.9414809978010444e+22


# Task 2

In [4]:
T=200
Na=5e16
Nd=3e17
Ea=hydrogen_like_energy(Si.eps, m=0.81*me)
Ed=Si.Eg - hydrogen_like_energy(Si.eps, m=0.36*me)
print(f'Ea: {Ea/eV}')
print(f'Ed: {(Si.Eg-Ed)/eV}')
print(f'Nc: {Si.Nc(T=T)}')
print(f'Nv: {Si.Nv(T=T)}')

pn = PNJunctionFullDepletion(Si, Na, Ea, Nd, Ed)
print(pn.n_mat.fermi_level(T=T)/eV)
print(pn.p_mat.fermi_level(T=T)/eV)
print(f'delta_phi: {pn.delta_phi(T)/volt}')
print(f'w_n: {pn.w_n(T)}')
print(f'w_p: {pn.w_p(T)}')
print(f'w: {pn.w(T)}')

pn = PNJunctionNonDegenerate(Si, Na, Ea, Nd, Ed)
Lp=2e-3
Ln=1e-2
Dp=12
Dn=36
j0_n = pn.j0_n(Dn, Ln)
j0_p = pn.j0_p(Dp, Lp)
((j0_n+j0_p)*from_unit(0.1, 'mm^2'))/unit('A')
ni2=(sqrt(Si.Nc(T=T)*Si.Nv(T=T))*exp(-Si.Eg/(2*k*T)))**2


p_n=ni2/(Si.Nc(T=T)*exp((pn.n_mat.fermi_level(T=T)-Si.Eg)/(k*T)))
n_p=ni2/(Si.Nv(T=T)*exp(-pn.p_mat.fermi_level(T=T)/(k*T)))
print(p_n, n_p)
print(e*Dp/Lp*p_n*from_unit(0.1, 'mm2')/unit('A'), e*Dn/Ln*n_p*from_unit(0.1, 'mm2')/unit('A'))
def cur(V):
    return e*Dn/Ln*n_p*from_unit(0.1, 'mm2')*(exp(e*V/k/T)-1)
def pwr(V):
    return V * cur(V)
pwr(3*volt)/unit('W')
v_max = bisect(lambda x: pwr(x)-from_unit(50, 'W'), 0.1*volt, 2*volt, xtol=0.01*volt)
i_max = cur(v_max)
print(i_max/unit('A'))

Ea: 0.08043349078490213
Ed: 0.03574821812662323
Nc: 2.9478007828414684e+18
Nv: 9.948827642089957e+18
1.0751378250122072
0.09524250030517577
delta_phi: 0.9798537051679436
w_n: 3.24534472744884e-06
w_p: 1.744075275274712e-05
w: 2.0686097480195958e-05
8.229303131918818e-09 4.530592611061823e-08
7.91034214995452e-27 2.6129943535472906e-26
63.426437463144865


# Task 3

In [5]:
# КДБ-1
sem = DopedSemiconductor.from_materials(Si, from_unit(450, 'cm / V s'), 'B', from_unit(1, 'Ohm cm'))
print(f'Na: {sem.Na:e}, Es: {sem.Ea/eV}, Ef: {sem.fermi_level()/eV}')

ms = MSJunction(Metal(4.2*eV), sem)
ms.delta_phi()/volt
df = ms.delta_phi()
n = sem.p_concentration()
depletion_width(Si.eps, n, -df)

Na: 1.387096e+16, Es: 0.045, Ef: 0.18277240753173832


2.7156364237350234e-05

# Task 4

In [6]:
def Eg(eps, a):
    return h_bar * sqrt(4*pi*32*e**2/((eps-1)*me*a**3))